In [8]:
import os
from graphviz import Digraph
import json

fillcolors = {
    'storage': 'blue',
    'hydropower': 'red',
    'instreamflowrequirement': 'green',
    'catchment': 'lightblue'
}

fontcolors = {
    'storage': 'white',
    'hydropower': 'white',
    'instreamflowrequirement': 'black'
}

dot = Digraph(comment='System')

for basin in ['stanislaus', 'merced', 'san_joaquin']:
    with open(os.path.join(basin, 'pywr_model.json')) as f:
        model = json.load(f)
        
    for node in model['nodes']:
        ntype = node['type'].lower()
        fillcolor = fillcolors.get(ntype, 'white')
        fontcolor = fontcolors.get(ntype, 'black')
        shape = 'rect' if ntype == 'storage' else 'oval'
        dot.node(node['name'], shape=shape,
                 style='filled' if fillcolor else None, fillcolor=fillcolor, fontcolor=fontcolor)

    dot.edges(model['edges'])
    
#     _dot = Digraph(name=basin)
#     _dot.edges(model['edges'])
#     _dot.render(os.path.join(basin, '{}_schematic.gv'.format(basin)), view=True)
    

connecting_edges = [
    ["San Joaquin River Outflow [node]", "San Joaquin-San Joaquin"],
    ["Merced River Outflow [node]", "Merced-San Joaquin"],
    ["STN-L-RIV2 Outflow [node]", "Stanislaus-San Joaquin"],
    ["San Joaquin-San Joaquin", "Merced-San Joaquin"],
    ["Merced-San Joaquin", "Stanislaus-San Joaquin"],
    ["Stanislaus-San Joaquin", "Delta"]
]
dot.edges(connecting_edges)
dot.render('system_schematic.gv', view=True)

'system_schematic.gv.pdf'